In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import duckdb
import polars as pl
import sys
sys.path.append('../') 
import upsert_utils

In [3]:
duckdb.sql("""
attach '../../../db.sqlite3';
use db;
""")

In [90]:
duckdb.sql("""
rollback;
""")

In [91]:
duckdb.sql("""
begin transaction;
""")

In [92]:
all_columns = duckdb.sql("""
select * from read_csv('ProdBattery.csv');
""").pl()

In [93]:
ob_dc_outputs = duckdb.sql(r"""
select
    csv_row_id,
    columns('^DCOutput__(.+)') as '\1',
from all_columns
""")
ob_dc_outputs

┌────────────┬───────────────────────────┬────────────────────────────┐
│ csv_row_id │ PowerDCContinuousMax_Unit │ PowerDCContinuousMax_Value │
│   int64    │          varchar          │           double           │
├────────────┼───────────────────────────┼────────────────────────────┤
│          1 │ kW                        │                        8.2 │
│          2 │ kW                        │                       5.76 │
│          3 │ kW                        │                      10.24 │
│          4 │ kW                        │                      20.48 │
│          5 │ kW                        │                      30.72 │
│          6 │ kW                        │                      40.96 │
│          7 │ kW                        │                       51.2 │
│          8 │ kW                        │                      61.44 │
│          9 │ kW                        │                      71.68 │
│         10 │ kW                        │                      

In [94]:
ob_prod_batteries = all_columns.drop('^.+__.+$')
duckdb.sql("""
describe ob_prod_batteries;
""")

┌─────────────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│         column_name         │ column_type │  null   │   key   │ default │  extra  │
│           varchar           │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ csv_row_id                  │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ ProdMfr_Value               │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ ProdCode_Value              │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Description_Value           │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ BatteryChemistryType_Value  │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ EnergyCapacityNominal_Unit  │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ EnergyCapacityNominal_Value │ DOUBLE      │ YES     │ NULL    │ NULL    │ NULL    │
└─────────────────────────────┴─────────────┴─────────

In [95]:
ob_prod_battery_ids = upsert_utils.insert_recursive(
    rows=duckdb.sql("""
    select
        csv_row_id,
        BatteryChemistryType_Value,
        EnergyCapacityNominal_Unit,
        EnergyCapacityNominal_Value,
    from ob_prod_batteries
    """),
    target_table_name='server_prodbattery',
    pk='product_ptr_id',
    fk_fills=[
        ('product_ptr_id', dict(
            rows=duckdb.sql("""
            select
                csv_row_id,
                replace(cast(uuidv4() as varchar), '-', '') as ProdID_Value,
                ProdMfr_Value,
                ProdCode_Value,
                coalesce(Description_Value, '') as Description_Value,
                'ProdBattery' as ProdType_Value,
            from ob_prod_batteries
            """),
            target_table_name='server_product',
            pk='id',
            superclass=True,
            fk_fills=[
                ('Dimension_id', dict(target_table_name='server_dimension', pk='id')),
            ]
        )),
        ('DCInput_id', dict(target_table_name='server_dcinput', pk='id')),
        ('DCOutput_id', dict(rows=ob_dc_outputs, target_table_name='server_dcoutput', pk='id')),
    ]
)
ob_prod_battery_ids

┌────────────┬────────────────┐
│ csv_row_id │ product_ptr_id │
│   int64    │     int64      │
├────────────┼────────────────┤
│          1 │          42221 │
│          2 │          42222 │
│          3 │          42223 │
│          4 │          42224 │
│          5 │          42225 │
│          6 │          42226 │
│          7 │          42227 │
│          8 │          42228 │
│          9 │          42229 │
│         10 │          42230 │
│          · │            ·   │
│          · │            ·   │
│          · │            ·   │
│        854 │          43074 │
│        855 │          43075 │
│        856 │          43076 │
│        857 │          43077 │
│        858 │          43078 │
│        859 │          43079 │
│        860 │          43080 │
│        861 │          43081 │
│        862 │          43082 │
│        863 │          43083 │
├────────────┴────────────────┤
│ 863 rows          2 columns │
│ (20 shown)                  │
└─────────────────────────────┘

In [96]:
all_columns = duckdb.sql("""
select * from read_csv('ProdCertification.csv')
""").pl()

In [97]:
ob_certification_agencies = duckdb.sql(r"""
select
    row_number() over () as csv_row_id,
    replace(cast(uuidv4() as varchar), '-', '') as CertificationAgencyID_Value,
    *,
from (
    select distinct
        columns('^CertificationAgency__(.+)') as '\1',
    from all_columns
    where CertificationAgencyName_Value not in (select CertificationAgencyName_Value from server_certificationagency)
)
""")
ob_certification_agencies

┌────────────┬──────────────────────────────────┬────────────────────────────────┐
│ csv_row_id │   CertificationAgencyID_Value    │ CertificationAgencyName_Value  │
│   int64    │             varchar              │            varchar             │
├────────────┼──────────────────────────────────┼────────────────────────────────┤
│          1 │ 11b19ae3d111422cb312fdecb108c82c │ SGS                            │
│          2 │ 899caefb85cb46e9a5cacbf621133373 │ TUV SUD America                │
│          3 │ 726b89bdf2f34c3b87404b57848ad70c │ CSA Group                      │
│          4 │ ba294206821c4ff897e87e1511fbbc88 │ UL                             │
│          5 │ 26d433b35307435290635d9d98db13ba │ TUV SUD                        │
│          6 │ 44ef9443176c4dfeae1281d39b09adf4 │ TUV                            │
│          7 │ c0509160537a479aa58e6dc0b35475ef │ TUV Rheinland                  │
│          8 │ db289620a5eb4322a69fc8ed1abd2810 │ TUV Rheinland of North America │
│   

In [98]:
upsert_utils.insert_recursive(
    rows=ob_certification_agencies,
    target_table_name='server_certificationagency',
    pk='entity_ptr_id',
    fk_fills=[
        ('entity_ptr_id', dict(target_table_name='server_entity', pk='id', superclass=True)),
    ],
)

┌────────────┬───────────────┐
│ csv_row_id │ entity_ptr_id │
│   int64    │     int64     │
├────────────┼───────────────┤
│          1 │             1 │
│          2 │             2 │
│          3 │             3 │
│          4 │             4 │
│          5 │             5 │
│          6 │             6 │
│          7 │             7 │
│          8 │             8 │
│          9 │             9 │
└────────────┴───────────────┘

In [99]:
ob_prod_certifications = duckdb.sql("""
select
    ob_prod_battery_ids.product_ptr_id,
    all_columns.* exclude(CertificationAgency__CertificationAgencyName_Value, prodbattery_id),
    server_certificationagency.entity_ptr_id as CertificationAgency_id,
from all_columns
join server_certificationagency on CertificationAgencyName_Value = CertificationAgency__CertificationAgencyName_Value
join ob_prod_battery_ids on prodbattery_id = ob_prod_battery_ids.csv_row_id
""")
ob_prod_certifications

┌────────────────┬─────────────────────────┬─────────────────────────────┬────────────┬────────────────────────┐
│ product_ptr_id │ CertificationDate_Value │ CertificationStandard_Value │ csv_row_id │ CertificationAgency_id │
│     int64      │          date           │           varchar           │   int64    │         int64          │
├────────────────┼─────────────────────────┼─────────────────────────────┼────────────┼────────────────────────┤
│          43083 │ 2024-11-27              │ UL1973_3_2022               │        863 │                      1 │
│          42932 │ 2025-10-31              │ UL1973_3_2022               │        712 │                      2 │
│          43069 │ 2022-12-27              │ UL1973_2_2018               │        849 │                      3 │
│          43080 │ 2023-01-05              │ UL1973_3_2022               │        860 │                      4 │
│          43021 │ 2023-01-04              │ UL1973_2_2018               │        801 │         

In [100]:
ob_prod_certification_ids = upsert_utils.insert_recursive(
    rows=duckdb.sql("""
    select * exclude(product_ptr_id) from ob_prod_certifications
    """),
    target_table_name='server_prodcertification',
    fk_fills=[
        ('Firmware_id', dict(
            target_table_name='server_firmware',
            pk='id',
            fk_fills=[
                ('Checksum_id', dict(target_table_name='server_checksum', pk='id')),
            ],
        )),
    ],
)
ob_prod_certification_ids

┌────────────┬───────┐
│ csv_row_id │  id   │
│   int64    │ int64 │
├────────────┼───────┤
│          1 │ 64467 │
│          2 │ 63998 │
│          3 │ 64353 │
│          4 │ 64351 │
│          5 │ 64349 │
│          6 │ 64347 │
│          7 │ 64345 │
│          8 │ 64343 │
│          9 │ 64341 │
│         10 │ 64339 │
│          · │   ·   │
│          · │   ·   │
│          · │   ·   │
│        854 │ 63620 │
│        855 │ 63612 │
│        856 │ 63629 │
│        857 │ 63633 │
│        858 │ 63625 │
│        859 │ 63617 │
│        860 │ 63608 │
│        861 │ 63621 │
│        862 │ 63613 │
│        863 │ 63605 │
├────────────┴───────┤
│      863 rows      │
│     (20 shown)     │
└────────────────────┘

In [101]:
upsert_utils.insert_recursive(
    rows=duckdb.sql("""
    select
        ob_prod_certification_ids.csv_row_id,
        product_ptr_id as product_id,
        ob_prod_certification_ids.id as prodcertification_id,
    from ob_prod_certification_ids
    join ob_prod_certifications using (csv_row_id)
    join ob_prod_battery_ids using (product_ptr_id)
    """),
    target_table_name='server_product_ProdCertifications',
)

┌────────────┬───────┐
│ csv_row_id │  id   │
│   int64    │ int64 │
├────────────┼───────┤
│        863 │ 63605 │
│        712 │ 63606 │
│        849 │ 63607 │
│        860 │ 63608 │
│        801 │ 63609 │
│        777 │ 63610 │
│        844 │ 63611 │
│        855 │ 63612 │
│        862 │ 63613 │
│        843 │ 63614 │
│          · │   ·   │
│          · │   ·   │
│          · │   ·   │
│         69 │ 64458 │
│         68 │ 64459 │
│         67 │ 64460 │
│         56 │ 64461 │
│         54 │ 64462 │
│         53 │ 64463 │
│         51 │ 64464 │
│         50 │ 64465 │
│         30 │ 64466 │
│          1 │ 64467 │
├────────────┴───────┤
│      863 rows      │
│     (20 shown)     │
└────────────────────┘

In [102]:
duckdb.sql("""
commit;
""")